In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import unary_union
import re

The first step is to load all of the datasets into dataframes for an initial inspection. Things to looks at include:
1. Do the same columns exist across all datasets?
2. Do duplicate columns exist across the datasets?
3. Where are the null values and how should they be managed?
4. Do data types need to be modified?

In [2]:
# Load the tree census data into a dataframe and display the first 5 rows
path_2015 = "../data/raw/tree_data/new_york_tree_census_2015.csv"
df = pd.read_csv(path_2015)
df.head()

tree_id  block_id  created_at  tree_dbh  stump_diam curb_loc status health  \
0   606945    305778  2016-06-28        10           0   OnCurb  Alive   Good   
1   160321    341273  2015-08-19         9           0   OnCurb  Alive   Good   
2   541347    325281  2015-12-30         7           0   OnCurb  Alive   Good   
3   613930    203822  2016-07-05        10           0   OnCurb  Alive   Good   
4    18353    338911  2015-06-13         4           0   OnCurb  Alive   Good   

                            spc_latin              spc_common  ... st_assem  \
0              Fraxinus pennsylvanica               green ash  ...       25   
1  Gleditsia triacanthos var. inermis             honeylocust  ...       34   
2                    Pyrus calleryana            Callery pear  ...       32   
3                    Pyrus calleryana            Callery pear  ...       46   
4                   Prunus virginiana  'Schubert' chokecherry  ...       31   

  st_senate   nta                  nta_name  boro_ct     state   latitude  \
0        14  QN37         Kew Gardens Hills  4125700  New York  40.724339   
1        13  QN28           Jackson Heights  4030902  New York  40.756626   
2        10  QN76              Baisley Park  4028800  New York  40.679777   
3        22  BK31                 Bay Ridge  3005000  New York  40.622743   
4        10  QN12  Hammels-Arverne-Edgemere  4095400  New York  40.596514   

   longitude          x_sp         y_sp  
0 -73.805180  1.038250e+06  203232.9417  
1 -73.894167  1.013571e+06  214953.6472  
2 -73.788463  1.042923e+06  187008.2671  
3 -74.037543  9.738279e+05  166160.5847  
4 -73.797622  1.040452e+06  156667.5017  

[5 rows x 41 columns]

In [3]:
# All of the custom functions will be stored in this cell for later use

def trim_and_lower(df):
  df2 = df.copy()

  df2.columns = [column.strip() for column in df2.columns]
  df2.columns = [column.strip().replace(" ", "_").lower() for column in df2.columns]

  df2 = df2.map(lambda x: x.replace("\xa0", " ") if isinstance(x, str) else x)
  df2 = df2.map(lambda x: x.strip() if isinstance(x, str) else x)
  df2 = df2.map(lambda x: x.lower() if isinstance(x, str) else x)

  return df2


def summary(df, cat_threshold=10, format_check=None):
    categorical = {}
    numerical = {}
    
    for col in df.columns:
        key = col
        value = df[col].nunique()
        labels = list(df[col].unique())
        if value <= cat_threshold:
            categorical.update({key: labels})
        else:
            numerical.update({key: value})

    nulls = null_columns(df)

    print(f"Columns containing unique values below or equal to the given threshold of {cat_threshold}:\n")
    for key, value in categorical.items():
        print(key,":", value)

    print(f"\n\nColumns containing unique values above the given threshold of {cat_threshold}:\n")
    for key, value in numerical.items():
        print(key,":", value)

    print("\n\nColumns with NULL values:\n")
    for key, value in nulls.items():
        print(key,":", value)

    print("\n\nIrregular format values:\n")
    if format_check:
        for col in format_check:
            len_set = set()
            df['length'] = df[col].astype("str").apply(len)
            min_len = df["length"].min()
            max_len = df["length"].max()
            len_set.update([min_len, max_len])
            if len(len_set) > 1:
                print(col, ":", len_set)
            df.drop(columns="length", inplace=True)
    else:
        print("None")

    

def null_columns(df):
    null_cols = df.columns[df.isnull().any()]
    df_null_cols = df[null_cols]
    return df_null_cols.isna().sum()


def compare_cols(df_list):
    cols_set = set()
    my_dict = {}

    for df in df_list:
        cols = df.columns
        cols_set.update(cols)

    for df in df_list:
        col_name = df.attrs["name"]
        search_result = []
        for item in cols_set:
            if item in df.columns:
                search_result.append(True)
            else:
                search_result.append(False)
        my_dict.update({col_name: search_result})

    new_df = pd.DataFrame(my_dict, index=(list(cols_set)))

    return new_df


def rename_columns(df_list, col_map):
    df_list_new = []
    for df in df_list:
        df = df.rename(columns=col_map)
        df_list_new.append(df)

    return df_list_new


def cols_set(df_list):
    cols_set = set()

    for df in df_list:
        cols = list(df.columns.sort_values())
        cols_set.update(cols)

    return cols_set


def get_locations_data_nearest(df, epsg=32618, max_distance=1800): # 4326
    
    # Load the geo data
    path_to_data = "../data/raw/geo_data/MODZCTA.geojson"
    gdf = gpd.read_file(path_to_data)[["modzcta", "geometry", "pop_est"]]

    # Filter out the row where modzcta == "99999"
    gdf = gdf.loc[gdf['modzcta'] != '99999']
    
    # Assign the new name to the chosen dataset
    locations_df = df.reset_index(drop=True)
    print("The shape of locations_df is:", locations_df.shape)
    
    # Convert locations_df to a GeoDataFrame
    geometry = [Point(xy) for xy in zip(locations_df['longitude'], locations_df['latitude'])]
    print("The shape of geometry is:", len(geometry))
    
    locations_gdf = gpd.GeoDataFrame(locations_df, geometry=geometry).reset_index(drop=True)
    print("The shape of locations_gdf is:", locations_gdf.shape)
    print("The CRS for locations_gdf is:", locations_gdf.crs)
    
    # Ensure both GeoDataFrames use the same coordinate reference system (CRS)
    locations_gdf.set_crs(epsg=4326, inplace=True)  # Assuming WGS84 CRS
    locations_gdf.to_crs(epsg=epsg, inplace=True)  # Assuming WGS84 CRS
    print("The CRS for locations_gdf is:", locations_gdf.crs)
    gdf.to_crs(epsg=epsg, inplace=True)
    print("The CRS for gdf is:", locations_gdf.crs)
    
    # Perform spatial join
    joined_gdf = gpd.sjoin_nearest(locations_gdf, gdf, how='left', max_distance=max_distance)
    print("The shape of joined_gdf is:", joined_gdf.shape)

    # Drop duplicates that may have been created during the spatial join
    joined_gdf_unique = joined_gdf.drop_duplicates(subset=["tree_id"], keep="first")
    print("The shape of joined_gdf_unique is:", joined_gdf_unique.shape)

    # Count null values in new "ZCTA5CE20" column
    nulls = joined_gdf_unique["modzcta"].isna().sum()
    print("The number of rows with null values for modzcta is:", nulls)
    print("\n****************************************\n")

    joined_gdf_unique = joined_gdf_unique.drop(columns="index_right").copy()

    return joined_gdf_unique

In [4]:
df.shape

(683788, 41)

In [5]:
# Check for null values
df.isna().sum()

tree_id            0
block_id           0
created_at         0
tree_dbh           0
stump_diam         0
curb_loc           0
status             0
health         31616
spc_latin      31619
spc_common     31619
steward       519438
guards        603922
sidewalk       31616
user_type          0
problems      457944
root_stone         0
root_grate         0
root_other         0
trunk_wire         0
trnk_light         0
trnk_other         0
brch_light         0
brch_shoe          0
brch_other         0
address            0
zipcode            0
zip_city           0
cb_num             0
borocode           0
boroname           0
cncldist           0
st_assem           0
st_senate          0
nta                0
nta_name           0
boro_ct            0
state              0
latitude           0
longitude          0
x_sp               0
y_sp               0
dtype: int64

## Fix 2015 dataset
- fill "nan" values in the "health" column with "dead" or "stump"
- "status" and "health" need to be standardised in the 2005 and 2015 datasets
- "steward" "nan" should be "none" and other values should be changed to "low", "medium" and "high" (not recorded for dead or stumps)
- fill "nan" values with "none" for "guards" (not recorded for dead or stumps)
- "sidewalk_damage" (yes / no): (not recorded for dead or stumps)
- some zip codes are of an irregular format, but it may be possible to predict them based on the lat. long. values, which are all correct

In [6]:
# Trim whitespace and convert all strings to lower case
df = trim_and_lower(df)

# Print a summary of the dataset using a custom function
summary(df, cat_threshold=20, format_check=["longitude", "latitude", "zipcode"])

Columns containing unique values below or equal to the given threshold of 20:

curb_loc : ['oncurb', 'offsetfromcurb']
status : ['alive', 'dead', 'stump']
health : ['good', 'fair', nan, 'poor']
steward : [nan, '1or2', '3or4', '4ormore']
guards : [nan, 'helpful', 'harmful', 'unsure']
sidewalk : ['nodamage', 'damage', nan]
user_type : ['treescount staff', 'volunteer', 'nyc parks staff']
root_stone : ['yes', 'no']
root_grate : ['no', 'yes']
root_other : ['no', 'yes']
trunk_wire : ['no', 'yes']
trnk_light : ['no', 'yes']
trnk_other : ['no', 'yes']
brch_light : ['no', 'yes']
brch_shoe : ['no', 'yes']
brch_other : ['no', 'yes']
borocode : [np.int64(4), np.int64(3), np.int64(1), np.int64(2), np.int64(5)]
boroname : ['queens', 'brooklyn', 'manhattan', 'bronx', 'staten island']
state : ['new york']


Columns containing unique values above the given threshold of 20:

tree_id : 683788
block_id : 101390
created_at : 483
tree_dbh : 146
stump_diam : 100
spc_latin : 132
spc_common : 132
problems : 23

In [7]:
# Drop duplicates based on the unique tree_id
df = df.drop_duplicates(subset="tree_id", keep='first').copy()

# Combine 'dead' and 'stump' into one value in the health column and fill null values with the mode
df["health"] = df.apply(lambda x: "dead / stump" if x["status"] in ["dead", "stump"] else x["health"], axis=1).fillna(df["health"].mode()[0])

# Replace the value for steward with 'not applicable' if the tree is 'dead' or 'stump' and fill nulls values with 'none'
df["steward"] = df.apply(lambda x: "not applicable" if x["status"] in ["dead", "stump"] else x["steward"], axis=1).fillna("none")

# Do the same for guards, sidewalk_damage, spc_common and spc_latin as is done for steward
df["guards"] = df.apply(lambda x: "not applicable" if x["status"] in ["dead", "stump"] else x["guards"], axis=1).fillna("none")
df["sidewalk_damage"] = df.apply(lambda x: "not applicable" if x["status"] in ["dead", "stump"] else ("yes" if x["sidewalk"] == "damage" else "no"), axis=1)
df["spc_common"] = df.apply(lambda x: "not applicable" if x["status"] in ["dead", "stump"] else x["spc_common"], axis=1)
df["spc_latin"] = df.apply(lambda x: "not applicable" if x["status"] in ["dead", "stump"] else x["spc_latin"], axis=1)

# Combine columns into simplified, single columns called ground_level_conflict and tree_level_conflict
ground_level_conflict_zipped = zip(df["root_stone"], df["root_grate"], df["root_other"])
df["ground_level_conflict"] = ["yes" if any(row == "yes" for row in values) else "no" for values in ground_level_conflict_zipped]
tree_level_conflict_zipped = zip(df["trnk_light"], df["trnk_other"], df["brch_light"], df["brch_shoe"], df["brch_other"], df["trunk_wire"])
df["tree_level_conflict"] = ["yes" if any(row == "yes" for row in values) else "no" for values in tree_level_conflict_zipped]

# Replace existing values in the stewardship column with more descriptive values
steward_replacements = {
    "1or2": "low",
    "3or4": "medium",
    "4ormore": "high"
}
df["steward"] = df["steward"].replace(steward_replacements)

# Rename the boroname column to borough
df.rename(columns={'boroname': 'borough'}, inplace=True)

# Drop unnecessary columns
cols_to_keep = [
    'ground_level_conflict',
    'guards',
    'health',
    'latitude',
    'longitude',
    'sidewalk_damage',
    'spc_common',
    'steward',
    'tree_dbh',
    'tree_id',
    'tree_level_conflict',
    "nta",
    "nta_name",
    "borough"
]

cols_to_drop = [col for col in df.columns if col not in cols_to_keep]
df = df.drop(columns=cols_to_drop).copy()

In [8]:
# Retrieve zip code and population estimate from the MODZCTA.geojson file
df = get_locations_data_nearest(df)

The shape of locations_df is: (683788, 14)
The shape of geometry is: 683788
The shape of locations_gdf is: (683788, 15)
The CRS for locations_gdf is: None
The CRS for locations_gdf is: EPSG:32618
The CRS for gdf is: EPSG:32618
The shape of joined_gdf is: (683788, 18)
The shape of joined_gdf_unique is: (683788, 18)
The number of rows with null values for modzcta is: 0

****************************************



In [9]:
col_map = {
    'diameter': "tree_diameter",
    'guards': "guards_impact",
    'health': "tree_condition",
    'spc_common': "species_common_name",
    'steward': "stewardship_level",
    'tree_dbh': "tree_diameter",
    'modzcta': "zip_code",
    'nta': "nta_code"
}

df = df.rename(columns=col_map)

In [10]:
df.columns

Index(['tree_id', 'tree_diameter', 'tree_condition', 'species_common_name',
       'stewardship_level', 'guards_impact', 'borough', 'nta_code', 'nta_name',
       'latitude', 'longitude', 'sidewalk_damage', 'ground_level_conflict',
       'tree_level_conflict', 'geometry', 'zip_code', 'pop_est'],
      dtype='object')

In [11]:
df.isna().sum()

tree_id                  0
tree_diameter            0
tree_condition           0
species_common_name      5
stewardship_level        0
guards_impact            0
borough                  0
nta_code                 0
nta_name                 0
latitude                 0
longitude                0
sidewalk_damage          0
ground_level_conflict    0
tree_level_conflict      0
geometry                 0
zip_code                 0
pop_est                  0
dtype: int64

In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer

# Create a filtered dataframe to fit the LabelEncoder
df_notna = df.loc[df.species_common_name.notna(), ["longitude", "latitude", "species_common_name"]]

# Initialize LabelEncoder and encode non-null values
le = LabelEncoder()
df_notna.species_common_name = le.fit_transform(df_notna.species_common_name)

# Update the original dataframe with the encoded values
df.update(df_notna.species_common_name)

# Initialize the KNNImputer and impute the missing values using the encoded values
imputer = KNNImputer(n_neighbors=2)
imputed_data = imputer.fit_transform(df[["longitude", "latitude", "species_common_name"]])

# Assign the imputed values back to species_common_name in the main dataframe
df['species_common_name'] = imputed_data[:, 2].astype(int)

# Transform the encoded values back to the original labels
df.species_common_name = le.inverse_transform(df.species_common_name)

In [13]:
df.isna().sum()

tree_id                  0
tree_diameter            0
tree_condition           0
species_common_name      0
stewardship_level        0
guards_impact            0
borough                  0
nta_code                 0
nta_name                 0
latitude                 0
longitude                0
sidewalk_damage          0
ground_level_conflict    0
tree_level_conflict      0
geometry                 0
zip_code                 0
pop_est                  0
dtype: int64

In [14]:
# Drop unnecessary columns
#df.drop(columns="index_right", inplace=True)

df.to_csv("../data/clean/tree_data/2015.csv", index=False)

#### Modify GEOJSON files

In [15]:
def combine_multipolygons_with_group(gdf, group_cols):

    # Save original CRS
    original_crs = gdf.crs

    # Group by the 'suburb' column and combine geometries
    grouped = gdf.groupby(group_cols)['geometry'].apply(lambda x: unary_union(x))
    
    # Create a new GeoDataFrame
    combined_gdf = gpd.GeoDataFrame(grouped, geometry='geometry').reset_index()

    # Ensure the GeoDataFrame has a CRS
    if combined_gdf.crs is None:
        combined_gdf.set_crs(original_crs, inplace=True)
    
    return combined_gdf


# Load your GeoDataFrame (example using a file)
path_to_data = "../data/raw/geo_data/2010 Census Tracts.geojson"
gdf = gpd.read_file(path_to_data)[["ntacode", "ntaname", "boro_name", "geometry"]]
gdf = trim_and_lower(gdf)

# List the columns to group by
group_cols = ["ntaname", "ntacode", "boro_name"]

# Combine MULTIPOLYGON geometries by suburb
combined_gdf = combine_multipolygons_with_group(gdf, group_cols)

# Convert to the appropriate CRS (was epsg=4326)
combined_gdf.to_crs(epsg=32618, inplace=True)

# Add area in m2 and km2
combined_gdf["area_nta_m2"] = combined_gdf.geometry.area
combined_gdf["area_nta_km2"] = combined_gdf["area_nta_m2"] / 1000000
combined_gdf["area_nta_hectares"] = combined_gdf["area_nta_m2"] / 10000 # hectares

# Add centroid for each MULTIPOLYGON
combined_gdf['nta_centroid'] = combined_gdf.geometry.centroid
combined_gdf['nta_centroid_lat'] = combined_gdf.nta_centroid.y
combined_gdf['nta_centroid_lon'] = combined_gdf.nta_centroid.x

new_col_names = {
    "ntaname": "nta_name",
    "ntacode": "nta_code",
    "boro_name": "borough",
    "geometry": "nta_geometry"
}
combined_gdf.rename(columns=new_col_names, inplace=True)

combined_gdf.drop(columns="nta_centroid", inplace=True)

# Export data to GeoJSON file
combined_gdf.to_file("../data/clean/geo_data/nta_data.geojson", index=False, driver="GeoJSON")

In [16]:
# Load the geo data
path_to_data = "../data/raw/geo_data/MODZCTA.geojson"
gdf = gpd.read_file(path_to_data)[["modzcta", "geometry"]]

# Reproject to the desired CRS for area calculation
gdf = gdf.to_crs(epsg=32618) # 3395
gdf["area_zip_m2"] = gdf["geometry"].area # meters squared
gdf["area_zip_km2"] = gdf["area_zip_m2"] / 1000000 # kilometers squared
gdf["area_zip_hectares"] = gdf["area_zip_m2"] / 10000 # hectares

# Calculate centroids in the projected CRS (EPSG:3395)
gdf['zip_centroid'] = gdf.geometry.centroid

# Reproject the centroids to EPSG:4326
#gdf = gdf.to_crs(epsg=4326)

# Extract latitude and longitude from the reprojected centroids
gdf['zip_centroid_lat'] = gdf.centroid.y
gdf['zip_centroid_lon'] = gdf.centroid.x

# Drop the intermediary centroid column if no longer needed
new_col_names = {
    'geometry': "zip_geometry",
    "modzcta": "zip_code"
}
df_zip_codes = gdf.rename(columns=new_col_names)

In [17]:
# TESTING CELL

# Load the geo data for the spatial join
gdf = combined_gdf[["nta_name", "nta_code", "borough", "nta_geometry"]].copy()
gdf.set_geometry("nta_geometry", inplace=True)

# Convert locations_df to a GeoDataFrame
#geometry = [Point(xy) for xy in zip(df_zip_codes['zip_centroid_lon'], df_zip_codes['zip_centroid_lat'])]
geometry = df_zip_codes.zip_centroid
zip_codes_gdf = gpd.GeoDataFrame(df_zip_codes, geometry=geometry)

# Ensure both GeoDataFrames use the same coordinate reference system (CRS)
zip_codes_gdf.set_crs(epsg=32618, inplace=True)  # Assuming WGS84 CRS
gdf = gdf.to_crs(epsg=32618)

# Perform spatial join
joined_gdf = gpd.sjoin_nearest(zip_codes_gdf, gdf, how='left', max_distance=350.0)
joined_gdf_unique = joined_gdf.drop_duplicates(subset=["zip_code"], keep="first")
#joined_gdf_unique.set_geometry("zip_geometry", inplace=True)
joined_gdf_unique["geometry"] = joined_gdf_unique["zip_geometry"]

# Trim all white space and convert to lower case for consistency
joined_gdf_unique = trim_and_lower(joined_gdf_unique)

# Drop unnecessary columns
joined_gdf_unique.drop(columns=["index_right", "zip_geometry"], inplace=True)

joined_gdf_unique.dtypes

zip_code               object
area_zip_m2           float64
area_zip_km2          float64
area_zip_hectares     float64
zip_centroid           object
zip_centroid_lat      float64
zip_centroid_lon      float64
geometry             geometry
nta_name               object
nta_code               object
borough                object
dtype: object

In [18]:
joined_gdf_unique.to_file("../data/clean/geo_data/zip_data.geojson", index=False, driver="GeoJSON")

In [19]:
borough_path = "../data/raw/geo_data/Borough Boundaries.geojson"
gdfb = gpd.read_file(borough_path)

# Convert boro_name to lower case and rename to borough
gdfb.boro_name = gdfb.boro_name.str.lower()
gdfb = gdfb.rename(columns={"boro_name": "borough"})

# Reproject to the desired CRS for area calculation
gdfb = gdfb.to_crs(epsg=32618) # 3395
gdfb["area_borough_m2"] = gdfb["geometry"].area # meters squared
gdfb["area_borough_km2"] = gdfb["area_borough_m2"] / 1000000 # kilometers squared
gdfb["area_borough_hectares"] = gdfb["area_borough_m2"] / 10000 # hectares

# Drop unnecessary columns
gdfb.drop(columns=["shape_area", "shape_leng"], inplace=True)

gdfb.to_file("../data/clean/geo_data/borough_data.geojson", index=False, driver="GeoJSON")